In [1]:
import os
import numpy as np
import pandas as pd
from scipy import signal
import pickle
from util import *

data = []
for number in range(1,23):
    
    # read data file
    name = 'peregrine_' +  "%02d" % number + '.xlsx'
    filename = '../rawdata/peregrine/' + name
    xlsx_data = pd.read_excel(filename, encoding="utf_8", sep=",")
    xlsx_data = np.array(xlsx_data)
    
    
    # get position
    # id 1: Peregrine
    px1 = xlsx_data[:,0]
    py1 = xlsx_data[:,1]
    pz1 = xlsx_data[:,2]

    # id 2: Target
    px2 = xlsx_data[:,4]
    py2 = xlsx_data[:,5]
    pz2 = xlsx_data[:,6]

    # degree to meter
    px1 = deg2meter(px1)
    py1 = deg2meter(py1)
    px2 = deg2meter(px2)
    py2 = deg2meter(py2)
    
    # low-pass filter
    samplerate, fp, fs, gpass, gstop = 5, 0.2, 2, 1, 2
    px1 = lowpass(px1, samplerate, fp, fs, gpass, gstop)
    py1 = lowpass(py1, samplerate, fp, fs, gpass, gstop)
    pz1 = lowpass(pz1, samplerate, fp, fs, gpass, gstop)
    px2 = lowpass(px2, samplerate, fp, fs, gpass, gstop)
    py2 = lowpass(py2, samplerate, fp, fs, gpass, gstop)
    pz2 = lowpass(pz2, samplerate, fp, fs, gpass, gstop)
    
    # shift target position to adjust GPS error
    px1_end = px1[-1]
    py1_end = py1[-1]
    pz1_end = pz1[-1]
    
    px2_end = px2[-1]
    py2_end = py2[-1]
    pz2_end = pz2[-1]
    
    disp_x = px1_end - px2_end
    disp_y = py1_end - py2_end
    disp_z = pz1_end - pz2_end
    
    px2 += disp_x 
    py2 += disp_y
    pz2 += disp_z
    
    # shift to origin
    x_min = np.min([np.min(px1), np.min(px2)])
    y_min = np.min([np.min(py1), np.min(py2)])
    z_min = np.min([np.min(pz1), np.min(pz2)])

    px1 -= x_min
    py1 -= y_min
    pz1 -= z_min
    px2 -= x_min 
    py2 -= y_min
    pz2 -= z_min

    # calculate velocity (difference)
    dpx1 = np.diff(px1)
    dpy1 = np.diff(py1)
    dpz1 = np.diff(pz1)
    dpx2 = np.diff(px2)
    dpy2 = np.diff(py2)
    dpz2 = np.diff(pz2)

    # ajust position data length to velocity data
    px1 = px1[:-1]
    py1 = py1[:-1]
    pz1 = pz1[:-1]
    px2 = px2[:-1]
    py2 = py2[:-1]
    pz2 = pz2[:-1]

    # get speed
    s1 = xlsx_data[:-1,3]
    s2 = xlsx_data[:-1,7]

    # append data
    data1 = []
    data1.append(dpx1); data1.append(dpy1); data1.append(dpz1)
    data1.append(px1); data1.append(py1); data1.append(pz1)
    data1.append(s1)
    
    data2 = []
    data2.append(dpx2); data2.append(dpy2); data2.append(dpz2)
    data2.append(px2); data2.append(py2); data2.append(pz2)
    data2.append(s2)
    
    data_seq = []
    data_seq.append(data1)
    data_seq.append(data2)
    
    data.append(data_seq)

# np.save('../peregrine_data', data)